
# AI Tutor for Student Success

This project is an AI tutor created to support newly admitted students as they begin their academic journey.  
It uses a Large Language Model (LLM) to understand students’ interests and provide personalized guidance and resources.  
As students progress, the tutor adapts by celebrating their successes and offering helpful tips when they struggle.  
The goal is to motivate students, reduce confusion, and help them feel confident and supported in school.

In [2]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
#constants
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [6]:
SYSTEM_PROMPT = (
    "You are a friendly tutor for newly admitted students. "
    "Be very simple and not overwhelming.\n"
    "For the given category, output exactly:\n"
    "Action: <one clear action step>\n"
    "Resource: <one helpful resource suggestion>\n"
    "Keep it short."
)

In [7]:
#Give advice to student
def tutor_category_advice(category_name, category_value, profile_summary):
    user_prompt = (
        f"Student summary (for context): {profile_summary}\n\n"
        f"Category: {category_name}\n"
        f"Student input: {category_value}\n\n"
        f"Give exactly 1 Action and 1 Resource."
    )

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2,
    )
    return response.choices[0].message.content.strip()


In [8]:
#collect input from student
def collect_list(prompt_text):
    items = []
    print(prompt_text + " (type 'done' to finish)")
    while True:
        userInput = input("> ").strip()
        if userInput.lower() == "done":
            break
        if userInput:
            items.append(userInput)
    return items

In [9]:
#Get the full information from student
def collect_profile():
    interests = collect_list("Enter interests (ex: CS, nursing, business)")
    goals = collect_list("Enter goals (ex: transfer, internship, GPA target)")

    level = input("Enter starting level (beginner/intermediate): ").strip()

    # Get more information that might prevent them to succeed
    time_per_week = input("Time available per week (hours): ").strip()
    language = input("Preferred language (ex: English, French): ").strip()
    job_hours = input("Job hours per week (0 if none): ").strip()

    learning_style = collect_list("Enter preferred learning style(s) (ex: videos, reading, practice)")

    profile = {
        "interests": interests,
        "goals": goals,
        "level": level,
        "constraints": {
            "time_per_week": time_per_week,
            "language": language,
            "job_hours": job_hours,
        },
        "learning_style": learning_style,
    }
    return profile

In [10]:
# Tutorial model to generate advice for student based on profile
def tutor_model(profile):
    summary = {
        "interests": profile["interests"],
        "goals": profile["goals"],
        "level": profile["level"],
        "constraints": profile["constraints"],
        "learning_style": profile["learning_style"],
    }

    categories = [
        ("Interests", profile["interests"]),
        ("Goals", profile["goals"]),
        ("Level", profile["level"]),
        ("Constraints", profile["constraints"]),
        ("Learning Style", profile["learning_style"]),
    ]

    sections = []
    for name, value in categories:
        advice = tutor_category_advice(name, value, summary)
        sections.append(f"### {name}\n{advice}")

    return "\n\n".join(sections)

In [ ]:
#Simulate a profile and test the model
profile = {
    "interests": ["Computer Science", "Web Development"],
    "goals": ["Transfer to a 4-year college", "Get an internship"],
    "level": "beginner",
    "constraints": {
        "time_per_week": 8,
        "language": "English",
        "job_hours": 20,
    },
    "learning_style": ["practice", "videos"],
}
print("\n--- Tutor Advice ---\n")
display(Markdown(tutor_model(profile)))